<a href="https://colab.research.google.com/github/sanaaria/Master-thesis/blob/main/Run_KG_GCN_DTIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [79]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [80]:
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [81]:
pip install ampligraph

In [82]:
import ampligraph as ampligraph

In [83]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd
import ampligraph as ampligraph
from ampligraph.datasets import load_from_csv

In [84]:
# from ampligraph.evaluation import train_test_split_no_unseen,generate_corruptions_for_fit
# # from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.datasets import load_from_csv


In [85]:
# from ampligraph.evaluation import evaluate_performance
# As of version 1.1.1, Ampligraph removed the 'evaluate_performance' function and instead introduced the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate metrics for evaluating model performance.
# If you are using version 2.0.1, you should be able to use the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate the desired metrics. Here's an example of how you can do this
from ampligraph.evaluation import mrr_score

In [86]:
from ampligraph.evaluation import mrr_score, hits_at_n_score ,mr_score

In [87]:
# from ampligraph.evaluation.common import generate_corruptions
from ampligraph.latent_features.layers.corruption_generation import CorruptionGenerationLayerTrain

In [88]:
from ampligraph.latent_features.layers.scoring import ComplEx

In [89]:
from ampligraph.latent_features.layers.scoring import TransE

In [90]:
from ampligraph.latent_features.layers.scoring import DistMult

In [91]:
# from ampligraph.utils import save_model,restore_model
from ampligraph.utils import save_model
from ampligraph.utils import restore_model

In [92]:
from tensorflow.python.keras.models import Model

In [93]:
from tensorflow.python.keras.layers import Dense, Dropout

In [94]:
from keras.layers import LSTM, Lambda, Layer, Dropout

In [95]:
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad,Adamax

In [96]:
from tensorflow import keras

In [97]:
#load data
################################################################

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
#data example: yamanishi_08
dt_08 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt',delimiter='\t',header=None)
# the script reads a csv file using pandas' read_csv function. This function reads the file from the specified path, which in this case is
# /content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt.

dt_08.columns = ['head','relation','tail']
# the columns of the DataFrame dt_08 are set using the columns attribute. The column names are 'head', 'relation', and 'tail'.

In [99]:
#kg
# ##This code is written in Python using the pandas library.
# #The goal of this code is to load two text files,
# which contain Knowledge Graph (KG) data, and concatenate them into a single pandas DataFrame.
# The KG data in these text files consists of triples (head, relation, tail), which are essentially edges in a graph.
# The 'head' is the subject, the 'relation' is the predicate, and the 'tail' is the object.

kg1 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/kegg_kg.txt',delimiter='\t',header=None)
# The pd.read_csv() function reads the specified file and creates a DataFrame. The delimiter='\t' argument tells pandas to use tabs as separators.
# The header=None argument tells pandas that the first row of the file does not contain column names.

kg2 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/yamanishi_uniprot_kg.txt',delimiter='\t',header=None)
#This code is similar to the previous one.

kg = pd.concat([kg1,kg2])
#Concatenate the two DataFrames.
#The pd.concat() function concatenates the input DataFrames into a single DataFrame.

kg.index = range(len(kg))
#Reset the index of the concatenated DataFrame.
#The index attribute of a DataFrame represents the index of the rows.
#This line of code resets the index of the concatenated DataFrame so that it starts from 0 and increments by 1.

kg.columns = ['head','relation','tail']
#Set the column names of the concatenated DataFrame.
#This line of code assigns new column names to the concatenated DataFrame.


#The resulting kg DataFrame contains the combined KG data from both text files.
# The DataFrame has three columns: 'head', 'relation', and 'tail'. The rows represent the triples (head, relation, tail) in the KG.

In [100]:
import matplotlib.pyplot as plt
import seaborn as sns

# # نمودار توزیع برای نوع داده‌ها
# sns.countplot(data=kg, x='relation')
# plt.xticks(rotation=90)
# plt.show()


In [101]:
pip install networkx matplotlib


In [102]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


In [103]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# اولین دو خط، دو شیء LabelEncoder را ایجاد می‌کند.
# این اشیاء برای تبدیل متغیرهای دسته‌ای به یک فرمت عددی که برای الگوریتم‌های یادگیری ماشین قابل فهم باشد، استفاده می‌شوند.
# تابع LabelEncoder() دو بار فراخوانی می‌شود تا دو شیء head_le و tail_le ایجاد شوند.
head_le = LabelEncoder()
tail_le = LabelEncoder()
relation_le=LabelEncoder()
# متد fit() بر روی هر دو شیء فراخوانی می‌شود. این متد پارامترهای لازم برای انجام رمزگذاری را محاسبه می‌کند.
head_le.fit(dt_08['head'].values)
tail_le.fit(dt_08['tail'].values)
relation_le.fit(dt_08['relation'].values)
# MinMaxScaler از ماژول preprocessing کتابخانه sklearn وارد می‌شود. این برای مقیاس‌بندی داده‌ها استفاده می‌شود.
mms = MinMaxScaler(feature_range=(0, 1))


In [104]:
dt_08['head'] = head_le.transform(dt_08['head'].values)
dt_08['tail'] = tail_le.transform(dt_08['tail'].values)
dt_08['relation']=relation_le.transform(dt_08['relation'].values)
print("نتیجه‌ی Label Encoding:")
print(dt_08)

نتیجه‌ی Label Encoding:
      head  relation  tail
0        0         0     0
1      181         0     0
2       11         0     1
3       60         0     1
4        5         0     3
...    ...       ...   ...
5122    55         0   943
5123    79         0   943
5124   335         0   943
5125   210         0   986
5126    63         0   987

[5127 rows x 3 columns]


In [105]:
dt_08 = mms.fit_transform(dt_08)
print("\nنتیجه‌ی مقیاس‌بندی با MinMaxScaler:")
print(dt_08)


نتیجه‌ی مقیاس‌بندی با MinMaxScaler:
[[0.         0.         0.        ]
 [0.22911392 0.         0.        ]
 [0.01392405 0.         0.00101215]
 ...
 [0.42405063 0.         0.95445344]
 [0.26582278 0.         0.99797571]
 [0.07974684 0.         0.99898785]]


In [106]:
#فراخوانی شناسه داروها (Drug IDs):
fp_id = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/791drug_struc.csv')['drug_id']

# فراخوانی شناسه پروتئین‌ها و توالی‌های آنها:
df_proseq = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/989proseq.csv')
df_proseq.columns = ['pro_id','pro_ids','seq']

# استخراج شناسه پروتئین‌ها:
pro_id = df_proseq['pro_id']

#فراخوانی ویژگی‌های داروها:
drug_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/morganfp.txt',delimiter=',')

#فراخوانی ویژگی‌های پروتئین‌ها:
pro_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/pro_ctd.txt',delimiter=',')

#مقیاس‌بندی ویژگی‌های پروتئین‌ها:
pro_feats_scaled = mms.fit_transform(pro_feats)

# کاهش ابعاد ویژگی‌های پروتئین با استفاده از PCA:
pro_feats_scaled2 = PCA(n_components=100).fit_transform(pro_feats_scaled)

#دوباره مقیاس‌بندی ویژگی‌های کاهش‌یافته:
pro_feats_scaled3 = mms.fit_transform(pro_feats_scaled2)

# ترکیب شناسه‌های داروها با ویژگی‌های آنها:
fp_df = pd.concat([fp_id,pd.DataFrame(drug_feats)],axis=1)

#ترکیب شناسه‌های پروتئین‌ها با ویژگی‌های آنها:
prodes_df = pd.concat([pro_id,pd.DataFrame(pro_feats_scaled3)],axis=1)


In [107]:
data_path = '/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10'


def load_data(i):
    # Read the train_fold csv file. The label is included.
    train = pd.read_csv(data_path+'/train_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Select only the positive examples (label == 1) from the train set.
    train_pos = train[train['label']==1]

    # Read the test_fold csv file. The label is included.
    test = pd.read_csv(data_path+'/test_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Merge the positive train examples and the knowledge graph into a single dataframe.
    data = pd.concat([train_pos,kg])[['head','relation','tail']]


    # Return the train, train_pos, test, and data dataframes.
    return train,train_pos,test,data



In [108]:
def roc_auc(y,pred):

  # این خط تابع roc_curve را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
 #تابع منحنی_roc منحنی ROC را برای مسئله طبقه بندی باینری داده شده، نرخ مثبت کاذب (FPR)، نرخ مثبت واقعی (TPR) و آستانه ها را محاسبه می کند.
    fpr, tpr, thresholds = metrics.roc_curve(y, pred)

    #roc_auc = metrics.auc(fpr, tpr): این خط تابع auc را از ماژول متریک با پارامترهای fpr و tpr فراخوانی می کند.
 # تابع auc مساحت زیر منحنی ROC را محاسبه می‌کند که امتیاز AUC-ROC است.
    roc_auc = metrics.auc(fpr, tpr)

    #return roc_auc: این خط امتیاز AUC-ROC محاسبه شده را برمی گرداند.
    return roc_auc


In [109]:
from sklearn import metrics

def aupr(y, pred):
    # این خط تابع precision_recall_curve را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع منحنی Precision-Recall را برای مسئله طبقه بندی باینری محاسبه می کند و دقت، بازیابی و آستانه ها را برمی گرداند.
    precision, recall, thresholds = metrics.precision_recall_curve(y, pred)

    # این خط تابع auc را از ماژول متریک با پارامترهای recall و precision فراخوانی می کند.
    # تابع auc مساحت زیر منحنی Precision-Recall را محاسبه می کند که امتیاز AUPR است.
    aupr_value = metrics.auc(recall, precision)

    # این خط امتیاز AUPR محاسبه شده را برمی گرداند.
    return aupr_value

In [110]:
from sklearn import metrics

def f1_score_custom(y, pred):
    # این خط تابع f1_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع F1 Score را برای مسئله طبقه بندی باینری محاسبه می کند.
    f1 = metrics.f1_score(y, pred)

    # این خط امتیاز F1 محاسبه شده را برمی گرداند.
    return f1

In [111]:
from sklearn import metrics

def precision_custom(y, pred):
    # این خط تابع precision_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع دقت را برای مسئله طبقه بندی باینری محاسبه می کند.
    precision = metrics.precision_score(y, pred)

    # این خط امتیاز دقت محاسبه شده را برمی گرداند.
    return precision

In [112]:
from sklearn import metrics

def accuracy_custom(y, pred):
    # این خط تابع accuracy_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع دقت را برای مسئله طبقه بندی باینری محاسبه می کند.
    accuracy = metrics.accuracy_score(y, pred)

    # این خط امتیاز دقت محاسبه شده را برمی گرداند.
    return accuracy

In [113]:
from sklearn import metrics

def mcc_custom(y, pred):
    # این خط تابع matthews_corrcoef را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع MCC را برای مسئله طبقه بندی باینری محاسبه می کند.
    mcc = metrics.matthews_corrcoef(y, pred)

    # این خط امتیاز MCC محاسبه شده را برمی گرداند.
    return mcc

In [114]:
from sklearn import metrics

def sensitivity_custom(y, pred):
    # این خط تابع recall_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع حساسیت (Sensitivity) را برای مسئله طبقه بندی باینری محاسبه می کند.
    sensitivity = metrics.recall_score(y, pred)

    # این خط امتیاز حساسیت محاسبه شده را برمی گرداند.
    return sensitivity


In [115]:
from sklearn import metrics

def specificity_custom(y, pred):
    # این خط ماتریس درهم‌ریختگی (Confusion Matrix) را با استفاده از y و pred محاسبه می‌کند.
    cm = metrics.confusion_matrix(y, pred)

    # این خط عناصر ماتریس درهم‌ریختگی را به ترتیب برای TN، FP، FN و TP استخراج می‌کند.
    tn, fp, fn, tp = cm.ravel()

    # این خط ویژگی (Specificity) را با استفاده از TN و FP محاسبه می‌کند.
    specificity = tn / (tn + fp)

    # این خط امتیاز ویژگی محاسبه شده را برمی گرداند.
    return specificity


In [116]:
def get_gcn_embeddings(model, train_triples, test_triples):
    # استخراج تعبیه‌های head از مدل برای داده‌های آموزشی و آزمایشی
    train_sub_embeddings = model.get_embeddings(train_triples['head'].values, embedding_type='e')
    test_sub_embeddings = model.get_embeddings(test_triples['head'].values, embedding_type='e')

    # استخراج تعبیه‌های tail از مدل برای داده‌های آموزشی و آزمایشی
    train_obj_embeddings = model.get_embeddings(train_triples['tail'].values, embedding_type='e')
    test_obj_embeddings = model.get_embeddings(test_triples['tail'].values, embedding_type='e')

    # ترکیب تعبیه‌های head و tail برای داده‌های آموزشی
    train_feats = np.concatenate([train_sub_embeddings, train_obj_embeddings], axis=1)

    # ترکیب تعبیه‌های head و tail برای داده‌های آزمایشی
    test_feats = np.concatenate([test_sub_embeddings, test_obj_embeddings], axis=1)

    # بازگرداندن تعبیه‌های آماده برای استفاده در GCN
    return train_feats, test_feats


In [ ]:

#به طور خلاصه، تابع get_features یک داده فریم داده، دو فریم داده fp_df و prodes_df و یک متغیر بولی use_pro می گیرد.
#ویژگی‌های دارویی را از fp_df و ویژگی‌های پروتئینی را از prodes_df با استفاده از ستون‌های سر و دم داده‌ها استخراج می‌کند.
#سپس، ویژگی ها را به صورت افقی بر اساس ارزش use_pro به هم متصل می کند. ماتریس ویژگی حاصل به عنوان خروجی تابع برگردانده می شود.
#این خط تابعی به نام get_features را تعریف می کند. به چهار پارامتر داده، fp_df، prodes_df و use_pro نیاز دارد.
def get_features(data,fp_df,prodes_df,use_pro):

    #این خط یک اتصال سمت چپ داده ها و fp_df را در ستون 'head' داده ها و ستون 'drug_id' در fp_df انجام می دهد.
 #پیوستن سمت چپ به این دلیل انجام می‌شود که می‌خواهیم همه رکوردها را از جدول سمت چپ (یعنی داده‌ها) و رکوردهای مطابقت‌شده را از جدول سمت راست (یعنی fp_df) نگه داریم.
 #سپس، کد 1025 ستون (از ستون 5 تا 1029) از دیتافریم حاصل را انتخاب می کند و با استفاده از ویژگی values ​​آن را به یک آرایه numpy تبدیل می کند.
 #نتیجه در متغیر_ویژگی های دارو ذخیره می شود.
    drug_features = pd.merge(data,fp_df,how='left',left_on='head',right_on='drug_id').iloc[:,4:1029].values

   #این خط عملیات مشابه قبلی را انجام می دهد، اما این بار داده ها را به هم می پیوندد و prodes_df را در ستون 'tail' داده و ستون 'pro_id' prodes_df را می پیوندد.
 #دوباره، کد 101 ستون (از ستون 5 تا 105) از دیتافریم حاصل را انتخاب می کند و با استفاده از ویژگی values ​​آن را به یک آرایه numpy تبدیل می کند.
 #نتیجه در متغیر pro_features ذخیره می شود.
    pro_features = pd.merge(data,prodes_df,how='left',left_on='tail',right_on='pro_id').iloc[:,4:105].values

   #این خط مقدار متغیر use_pro را بررسی می کند. اگر True باشد، با استفاده از تابع np.concatenate، drug_features و pro_features را به صورت افقی به هم متصل می کند.
 #اگر use_pro False باشد، مستقیماً ویژگی‌های دارو را به ویژگی متغیر اختصاص می‌دهد.
    if use_pro:
        feature = np.concatenate([drug_features,pro_features],axis=1)
    else:
        feature = drug_features

    #این خط ماتریس ویژگی نهایی را برمی گرداند.
    return feature
